In [1]:
import pandas as pd
import numpy as np

# Load Data

In [29]:
reagent_data = pd.read_csv('/Users/alanwortman/Desktop/Well Plate Example.csv')
reagent_data
#Note cant, put coorinates into excel as (0,0) because it will read as a string and not a tuple

,location,x,y,chemical_name,class,smiles
0,A1,0.0,0.0,PC1,photocatalyst,hi
1,A2,4.5,0.0,PC2,photocatalyst,hi
2,A3,9.0,0.0,PC3,photocatalyst,hi
3,A4,13.5,0.0,NaN,NaN,NaN
4,A5,18.0,0.0,NaN,NaN,NaN
...,...,...,...,...,...,...
379,P20,85.5,67.5,NaN,NaN,NaN
380,P21,90.0,67.5,NaN,NaN,NaN
381,P22,94.5,67.5,NaN,NaN,NaN
382,P23,99.0,67.5,NaN,NaN,NaN


# Create Function to Get coordinates

In [25]:
#want to get the x & y for each chemical and place them into a new column together as a tuple
#any time the table is updated, you must re-run this cell
coordinates=[]
def getx_y(dataframe):
    for i in reagent_data.index:
        coordinates.append((reagent_data.x[i], reagent_data.y[i]))
    
def create_coordinates(dataframe): #this is a more updated version...use this one
    coordinates = []
    for i in reagent_data.index:
        coordinates.append((reagent_data.x[i], reagent_data.y[i]))
    dataframe['coordinates'] = coordinates    

# Add Coordinates to Dataframe

In [36]:
#These coordinates are in mm
#this is an older version of the code which was called separately (before create_coordinates was written)
getx_y(reagent_data)
reagent_data['coordinates'] = coordinates
reagent_data

,location,x,y,chemical_name,class,smiles,coordinates
0,A1,0.0,0.0,PC1,photocatalyst,hi,"(0.0, 0.0)"
1,A2,4.5,0.0,PC2,photocatalyst,hi,"(4.5, 0.0)"
2,A3,9.0,0.0,PC3,photocatalyst,hi,"(9.0, 0.0)"
3,A4,13.5,0.0,NaN,NaN,NaN,"(13.5, 0.0)"
4,A5,18.0,0.0,NaN,NaN,NaN,"(18.0, 0.0)"
...,...,...,...,...,...,...,...
379,P20,85.5,67.5,NaN,NaN,NaN,"(85.5, 67.5)"
380,P21,90.0,67.5,NaN,NaN,NaN,"(90.0, 67.5)"
381,P22,94.5,67.5,NaN,NaN,NaN,"(94.5, 67.5)"
382,P23,99.0,67.5,NaN,NaN,NaN,"(99.0, 67.5)"


# Drop All Cells With No Entries

In [35]:
#Note: All descriptors must be filled out or the entry will be dropped by the .dropna() function
# reagent_data_updated = reagent_data.dropna().reset_index(drop = True) ...this is the old version
# reagent_data_updated

create_coordinates(reagent_data)
reagent_data_updated = reagent_data.dropna().reset_index(drop = True)
reagent_data_updated
# reagent_data_updated.to_dict()

,location,x,y,chemical_name,class,smiles,coordinates
0,A1,0.0,0.0,PC1,photocatalyst,hi,"(0.0, 0.0)"
1,A2,4.5,0.0,PC2,photocatalyst,hi,"(4.5, 0.0)"
2,A3,9.0,0.0,PC3,photocatalyst,hi,"(9.0, 0.0)"
3,B1,0.0,4.5,B1,base,i,"(0.0, 4.5)"
4,B2,4.5,4.5,B2,base,S,"(4.5, 4.5)"
5,B3,9.0,4.5,B3,base,D,"(9.0, 4.5)"
6,B4,13.5,4.5,B4,base,F,"(13.5, 4.5)"
7,C1,0.0,9.0,R1,reagent,s,"(0.0, 9.0)"
8,C2,4.5,9.0,R2,reagent,a,"(4.5, 9.0)"


# Make New Dataframe Where Each Class Type is a Column


In [163]:
# Want to see {'photocatalysts': [pc1, pc2, etc.], 'bases': [b1, b2, etc.], 'reagents': [r1, r2, etc.]} as a dataframe

In [164]:
#Then want to replace the pc1, pc2, etc. in each column with the coordinates
#The idea is to then use the pandas series of the coordinate tuples for the g-code generator developed before

In [6]:
classes = reagent_data_updated['class'].unique() #this is a numpy array
for clas in classes:
    print(clas)

photocatalyst
base
reagent


In [37]:
'''for a useful link see:
https://stackoverflow.com/questions/65262268/how-do-i-turn-categorical-column-values-into-different-column-names'''

new_df = reagent_data_updated.set_index(
    [reagent_data_updated.groupby('class').cumcount(),
     'class'])['chemical_name'].unstack()

new_df

class,base,photocatalyst,reagent
0,B1,PC1,R1
1,B2,PC2,R2
2,B3,PC3,NaN
3,B4,NaN,NaN


In [10]:
new_dataframe = new_df.replace(to_replace = 'B1', value = 'hi')
new_dataframe

class,base,photocatalyst,reagent
0,hi,PC1,R1
1,B2,PC2,R2
2,B3,PC3,NaN
3,B4,NaN,NaN


In [11]:
#Write function to replace reagent names with their coordinates

In [42]:
new_dfA = reagent_data_updated.set_index(
    [reagent_data_updated.groupby('class').cumcount(),
     'class'])['chemical_name'].unstack()

new_dfA

class,base,photocatalyst,reagent
0,B1,PC1,R1
1,B2,PC2,R2
2,B3,PC3,NaN
3,B4,NaN,NaN


In [48]:
# def get_columns(df):
#     for value in df.columns.values:
#         print(df.value)
        
def get_columns(df):
    column_headers = list(df.columns.values)
    print(column_headers)
get_columns(new_dfA)

['base', 'photocatalyst', 'reagent']


In [49]:
def get_columns(df):
    column_headers = list(df.columns.values)
    for header in column_headers:
        output_data = {"chemical_name":[], "x":[], "y":[]}
        
get_columns(new_dfA)

base
photocatalyst
reagent


In [88]:
letters = ['A', 'B', 'C']
wells = [reagent_data_updated[reagent_data_updated['location'].str.startswith(l)] for l in letters]
print(wells)


[  location    x    y chemical_name          class smiles coordinates
0       A1  0.0  0.0           PC1  photocatalyst     hi  (0.0, 0.0)
1       A2  4.5  0.0           PC2  photocatalyst     hi  (4.5, 0.0)
2       A3  9.0  0.0           PC3  photocatalyst     hi  (9.0, 0.0),   location     x    y chemical_name class smiles  coordinates
3       B1   0.0  4.5            B1  base      i   (0.0, 4.5)
4       B2   4.5  4.5            B2  base      S   (4.5, 4.5)
5       B3   9.0  4.5            B3  base      D   (9.0, 4.5)
6       B4  13.5  4.5            B4  base      F  (13.5, 4.5),   location    x    y chemical_name    class smiles coordinates
7       C1  0.0  9.0            R1  reagent      s  (0.0, 9.0)
8       C2  4.5  9.0            R2  reagent      a  (4.5, 9.0)]
